In [1]:
import pandas as pd
from tkinter import *
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from sklearn.linear_model import LinearRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg, NavigationToolbar2Tk)

In [13]:
def get_data():
    data=pd.read_csv(r"C:\Users\zafreen\Desktop\Zoha\BDF\BDFSWA\Leaders_data.csv")
    data=data.drop_duplicates(subset='Requisition ID')
    data['Requisition Approved Date']=pd.to_datetime(data['Requisition Approved Date'])
    data=data[data['Requisition Approved Date']<="2021-04-30"]
    data['Year']=data['Requisition Approved Date'].dt.year
    data['Month']=data['Requisition Approved Date'].dt.month
    data=data[data['Requisition Current Status']=='Filled']
    data=data.drop(columns=['Requisition Current Status'])
    data=data.reset_index(drop=True)
    return data

In [14]:
def TA_match(leadername,ron,rod,c_n,jf,rt,ro,rjg,rjc,sn,cn,date):
    full=get_data()
    full=full[full['Current Manager 03 Email']==leadername]
    full=full.reset_index(drop=True)
    df=full
    collection = pd.DataFrame()
    recruiters = df['Recruiter Email'].unique()
    for i in range(len(recruiters)):
    #     print(i)
        collection.loc[i,'Recruiter'] = recruiters[i]
        temp = df[df['Recruiter Email'] == recruiters[i]]
        temp=temp.drop(['Recruiter Email','Requisition ID'],axis=1)
        temp['col'] = temp.apply(lambda row: ' '.join(row.values.astype(str)), axis = 1)
        collection.loc[i,'Features'] = temp['col'].astype(str).str.cat(sep = ' ')
#     active = pd.read_csv(r"C:\Users\zafreen\Downloads\datasets\hc.csv")
#     for i in range(len(collection)):
#     #     print(i)
#         if active['Email.Address'].str.contains(collection.loc[i,'Recruiter']).any():  
#             j = active['Email.Address'].index[(active['Email.Address'] == collection.loc[i,'Recruiter'])].tolist()[0]
#             if active.loc[j,'Working.Status'] == 'Active':
#                  collection.loc[i,'Active'] = 1
#             else:
#                 collection.loc[i,'Active'] = 0
#                 continue
#         collection.loc[i,'Active'] = 1
#     collection=collection[collection['Active'] == 1].reset_index(drop=True)
#     del collection['Active']
    collection['Features']=collection['Features'].str.lower()
    schedule = pd.read_csv(r"C:\Users\zafreen\Desktop\Zoha\BDF\NA\Auto Recommend Recruiter\Schedule.csv")
    terms = list(set(collection['Features'].str.cat(sep = ' ').split(" ")))
    import math
    matrix = [[0 for i in range(len(collection))]for j in range(len(terms))]
    N = len(collection)
    doc_freq = []
    for t in terms:
        count = 0
        for d in collection['Features']:
            if t in d:
                count += 1
        doc_freq.append(count)
    for q in range(len(terms)):
        for d in range(len(collection['Features'])):
            if terms[q] in collection.loc[d,'Features']:
                matrix[q][d] = collection.loc[d,'Features'].count(terms[q])*math.log(N/doc_freq[q]) 
    def calculatesim(ron,rod,c_n,jf,rt,ro,rjg,rjc,sn,cn,date):
        query=ron+" "+rod+" "+c_n+" "+jf+" "+rt+" "+ro+" "+rjg+" "+rjc+" "+sn+" "+cn
        query = query.lower()
        q = query.split(" ")
#         date = datetime.strptime(date, '%d/%m/%y')
        date=date
        score = [0 for i in range(len(terms))]
        recruiters = collection['Recruiter']
        for t in q:
            if t in terms:
                i = terms.index(t)
                score[i] = q.count(t)*math.log(N/doc_freq[i])
        sim = [0 for i in range(len(recruiters))]
        q_norm = 0
        for i in score:
            q_norm += pow(i,2)
        q_norm = math.sqrt(q_norm)
        for d in range(len(recruiters)):
            num = 0
            den = 0
            for t in range(len(terms)):
                num += matrix[t][d]*score[t]
                den += pow(matrix[t][d],2)
            sim[d] = num/(math.sqrt(den)+q_norm)
        matches = []
        i=0
        while i<5:
            ind=sim.index(max(sim))
            if sim[ind]==0:
                return matches
            if recruiters[ind] == 'unspecified':
                sim[ind]=0
                continue
            if schedule['Recruiter E-mail'].str.contains(recruiters[ind]).any():
                j=schedule['Recruiter E-mail'].index[(schedule['Recruiter E-mail'] == recruiters[ind])].tolist()
                dates=[]
                for k in j:
                    dates.append(datetime.strptime(schedule.loc[k,'Requisition Date Entered'],'%d/%b/%y'))
                if date in dates:
                    sim[ind]=0
                    continue            
            matches.append(recruiters[ind])
            sim[ind]=0
            i=i+1
        return matches
    result=calculatesim(ron,rod,c_n,jf,rt,ro,rjg,rjc,sn,cn,date)
    return result


In [15]:
TA_match('t.k.anand@oracle.com','5RD1 - Oracle Analytics Cloud (OAC) - ORCL USA','0BPH','10540.SWENG.IC4','IC4','PRODEV','Principal Member of Technical Staff (JoinOCI-Platform)','New Position','United States','Redwood city','California','2019-14-06')

AttributeError: 'DataFrame' object has no attribute 'subset'

In [ ]:
import tkinter as tk
from PIL import Image,  ImageTk
root_1=tk.Tk()
root_1.title("Demand Forecasting-Recruiter Match Application")
root_1.geometry('1350x1000')
root_1.iconbitmap('oracle1.ico')
root_1.config(bg='white')
username=tk.StringVar()
requisition_organization_name = tk.StringVar()
requisition_organization_department = tk.StringVar()
requisition_country_name =tk.StringVar()
requisition_job_function = tk.StringVar()
requisition_title = tk.StringVar()
requisition_type_of_opening = tk.StringVar()
requisition_job_code = tk.StringVar()
requisition_job_grade = tk.StringVar()
requisition_state_name = tk.StringVar()
requisition_city_name = tk.StringVar()
requisition_hm=tk.StringVar()
requisition_approved_date=tk.StringVar()
def Predict():
    global User
    User=username.get()
    organization_name =requisition_organization_name.get()
    organization_department =requisition_organization_department.get()
    country_name =requisition_country_name.get()
    job_function =requisition_job_function.get()
    title =requisition_title.get()
    type_of_opening =requisition_type_of_opening.get()
    job_code =requisition_job_code.get()
    job_grade =requisition_job_grade.get()
    state_name =requisition_state_name.get()
    city_name =requisition_city_name.get()
#     hm=requisition_hm.get()
    approved_date=requisition_approved_date.get()
    Predicted_TA=TA_match(User,organization_name,organization_department,country_name,job_function,title,type_of_opening,job_code,job_grade,state_name,city_name,approved_date)
    Predicted_TA_join = "\n".join(Predicted_TA)
    Prediction=tk.Label(root_1, text=Predicted_TA_join,bg='white')
    Prediction.grid(row=13,column=3)
    print(Predicted_TA)
Username_label = tk.Label(root_1, text = 'Email', font=('calibre',10, 'bold'),bg='white')
Username_entry = tk.Entry(root_1,textvariable =username, font=('calibre',10,'normal'),bg='white')
requisition_organization_department_label = tk.Label(root_1, text = 'Requisition Organisation Department', font=('calibre',10, 'bold'),bg='white')
requisition_organization_department_entry = tk.Entry(root_1,textvariable = requisition_organization_department, font=('calibre',10,'normal'),bg='white')
requisition_organization_name_label = tk.Label(root_1, text = 'Requisition Organisation Name', font=('calibre',10, 'bold'),bg='white')
requisition_organization_name_entry = tk.Entry(root_1,textvariable = requisition_organization_name, font=('calibre',10,'normal'),bg='white')
requisition_country_name_label = tk.Label(root_1, text = 'Requisition Country Name', font=('calibre',10, 'bold'),bg='white')
requisition_country_name_entry = tk.Entry(root_1,textvariable = requisition_country_name, font=('calibre',10,'normal'),bg='white')
requisition_job_function_label = tk.Label(root_1, text = 'Requisition Job Function', font=('calibre',10, 'bold'),bg='white')
requisition_job_function_entry = tk.Entry(root_1,textvariable = requisition_job_function, font=('calibre',10,'normal'),bg='white')
requisition_title_label = tk.Label(root_1, text = 'Requisition Job Title', font=('calibre',10, 'bold'),bg='white')
requisition_title_entry = tk.Entry(root_1,textvariable = requisition_title, font=('calibre',10,'normal'),bg='white')
requisition_type_of_opening_label = tk.Label(root_1, text = 'Requisition type of Opening', font=('calibre',10, 'bold'),bg='white')
requisition_type_of_opening_entry = tk.Entry(root_1,textvariable =  requisition_type_of_opening, font=('calibre',10,'normal'),bg='white')
requisition_job_code_label = tk.Label(root_1, text = 'Requisition job code', font=('calibre',10, 'bold'),bg='white')
requisition_job_code_entry = tk.Entry(root_1,textvariable =  requisition_job_code, font=('calibre',10,'normal'),bg='white')
# requisition_hm_label = tk.Label(root, text = 'Requisition HM', font=('calibre',10, 'bold'),bg='white')
# requisition_hm_entry = tk.Entry(root,textvariable =requisition_hm, font=('calibre',10,'normal'),bg='white')
requisition_job_grade_label = tk.Label(root_1, text = 'Requisition job grade', font=('calibre',10, 'bold'),bg='white')
requisition_job_grade_entry = tk.Entry(root_1,textvariable =   requisition_job_grade, font=('calibre',10,'normal'),bg='white')
requisition_state_name_label = tk.Label(root_1, text = 'Requisition State', font=('calibre',10, 'bold'),bg='white')
requisition_state_name_entry = tk.Entry(root_1,textvariable =requisition_state_name, font=('calibre',10,'normal'),bg='white')
requisition_city_name_label = tk.Label(root_1, text = 'Requisition city', font=('calibre',10, 'bold'),bg='white')
requisition_city_name_entry = tk.Entry(root_1,textvariable =requisition_city_name, font=('calibre',10,'normal'),bg='white')
requisition_approved_date_label = tk.Label(root_1, text = 'Requisition Date', font=('calibre',10, 'bold'),bg='white')
requisition_approved_date_entry = tk.Entry(root_1,textvariable =requisition_approved_date, font=('calibre',10,'normal'),bg='white')
predict_button=tk.Button(root_1,text = 'Predict ', command = Predict)
Username_label.grid(row=0,column=0)
Username_entry.grid(row=0,column=1)
requisition_organization_department_label.grid(row=5,column=0)
requisition_organization_department_entry.grid(row=5,column=1)
requisition_organization_name_label.grid(row=6,column=0)
requisition_organization_name_entry.grid(row=6,column=1)
requisition_job_code_label.grid(row=7,column=0)
requisition_job_code_entry.grid(row=7,column=1)
requisition_job_grade_label.grid(row=8,column=0)
requisition_job_grade_entry.grid(row=8,column=1)
requisition_job_function_label.grid(row=9,column=0)
requisition_job_function_entry.grid(row=9,column=1)
requisition_title_label.grid(row=10,column=0)
requisition_title_entry.grid(row=10,column=1)
requisition_type_of_opening_label.grid(row=5,column=2)
requisition_type_of_opening_entry.grid(row=5,column=3)
# requisition_hm_label.grid(row=3,column=2)
# requisition_hm_entry.grid(row=3,column=3)
requisition_country_name_label.grid(row=6,column=2)
requisition_country_name_entry.grid(row=6,column=3)
requisition_state_name_label.grid(row=7,column=2)
requisition_state_name_entry.grid(row=7,column=3)
requisition_city_name_label.grid(row=8,column=2)
requisition_city_name_entry.grid(row=8,column=3)
requisition_approved_date_label.grid(row=9,column=2)
requisition_approved_date_entry.grid(row=9,column=3)
predict_button.grid(row=10,column=3)
root_1.mainloop()

['PETER.HARTZELL@ORACLE.COM', 'KEVIN.M.MCGUIRE@ORACLE.COM', 'KATRINA.TRUSSLER@ORACLE.COM', 'IVY.SEVIGNY@ORACLE.COM', 'LUCAS.JARAMILLO@ORACLE.COM']
['SHERRY.OLIVER@ORACLE.COM', 'PETER.HARTZELL@ORACLE.COM', 'STEPHEN.B.EDWARDS@ORACLE.COM', 'JOHN.J.THOMPSON@ORACLE.COM', 'KEVIN.M.MCGUIRE@ORACLE.COM']
